# Großtext-Analyse mit Limits

Diese Notebook implementiert eine Lösung zur Analyse großer Textdateien unter Berücksichtigung von Speicherlimits.

## Funktionalität
- Verarbeitung großer Textdateien in Chunks
- Speicherüberwachung und -management
- Fortschrittsanzeige während der Verarbeitung
- Konfigurierbare Chunk-Größe und Speichergrenzen

## Limits
- Maximale Chunk-Größe: 100MB
- Speicherlimit: 80% der verfügbaren RAM
- Maximale Dateigröße: Abhängig vom verfügbaren Speicher

In [ ]:
import os
import sys
import psutil
import threading
from tqdm import tqdm
from typing import Generator, Any

# Konstanten
MAX_CHUNK_SIZE = 1024 * 1024 * 100  # 100MB
MEMORY_LIMIT_PERCENT = 80  # 80% des verfügbaren RAM

In [ ]:
class TextChunkProcessor:
    def __init__(self, chunk_size: int = MAX_CHUNK_SIZE):
        self.chunk_size = chunk_size
        self._memory_monitor = threading.Thread(target=self._monitor_memory, daemon=True)
        self.should_stop = False

    def process_file(self, filepath: str) -> Generator[str, Any, None]:
        file_size = os.path.getsize(filepath)

        with open(filepath, 'r') as file:
            with tqdm(total=file_size, unit='B', unit_scale=True) as pbar:
                chunk = file.read(self.chunk_size)
                while chunk and not self.should_stop:
                    yield chunk
                    pbar.update(len(chunk))
                    chunk = file.read(self.chunk_size)

    def _monitor_memory(self):
        while not self.should_stop:
            if self._get_memory_usage() > MEMORY_LIMIT_PERCENT:
                self.should_stop = True
            time.sleep(1)

    @staticmethod
    def _get_memory_usage() -> float:
        return psutil.Process().memory_percent()

In [ ]:
def analyze_large_text(filepath: str) -> dict:
    """
    Hauptfunktion zur Analyse großer Textdateien
    """
    results = {
        'word_count': 0,
        'line_count': 0,
        'char_count': 0,
        'memory_usage': []
    }

    processor = TextChunkProcessor()
    processor._memory_monitor.start()

    try:
        for chunk in processor.process_file(filepath):
            results['word_count'] += len(chunk.split())
            results['line_count'] += chunk.count('\n')
            results['char_count'] += len(chunk)
            results['memory_usage'].append(processor._get_memory_usage())

            if processor.should_stop:
                print("Warnung: Speicherlimit erreicht!")
                break

    finally:
        processor.should_stop = True
        processor._memory_monitor.join()

    return results

In [ ]:
def clear_memory():
    """
    Speicherbereinigung durchführen
    """
    import gc
    gc.collect()

def get_available_memory() -> float:
    """
    Verfügbaren Speicher ermitteln
    """
    return psutil.virtual_memory().available / (1024 * 1024)  # MB

def check_file_processable(filepath: str) -> bool:
    """
    Prüft, ob die Datei verarbeitet werden kann
    """
    file_size = os.path.getsize(filepath)
    available_memory = get_available_memory()
    return file_size < (available_memory * (MEMORY_LIMIT_PERCENT / 100))

In [ ]:
# Beispielverwendung
if __name__ == "__main__":
    beispiel_datei = "große_textdatei.txt"

    if check_file_processable(beispiel_datei):
        ergebnisse = analyze_large_text(beispiel_datei)
        print(f"Wortanzahl: {ergebnisse['word_count']}")
        print(f"Zeilenanzahl: {ergebnisse['line_count']}")
        print(f"Zeichenanzahl: {ergebnisse['char_count']}")
        print(f"Durchschnittliche Speicherauslastung: {sum(ergebnisse['memory_usage'])/len(ergebnisse['memory_usage']):.2f}%")
    else:
        print("Fehler: Datei zu groß für verfügbaren Speicher")